In [1]:
import torch
from torchvision import datasets, transforms

# Define the transformation (converting to tensor)
transform = transforms.ToTensor()

# Download the MNIST training dataset
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)

# Download the MNIST test dataset
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

print("MNIST dataset downloaded successfully.")


/opt/anaconda3/envs/GPSG/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 9912422/9912422 [03:48<00:00, 43434.39it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 28881/28881 [00:00<00:00, 102507.52it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 1648877/1648877 [00:36<00:00, 44801.92it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)>



100%|██████████| 4542/4542 [00:00<00:00, 1002448.37it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw

MNIST dataset downloaded successfully.


In [139]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations: converting to tensor and normalizing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # Normalize to mean=0.1307 and std=0.3081 for MNIST
])

# Download and apply transforms to MNIST datasets
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# Create DataLoaders to batch, shuffle (for training), and load the data efficiently
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Example to iterate over the training data
for batch_idx, (data, target) in enumerate(train_loader):
    print("Batch index:", batch_idx)
    print("Data shape:", data.shape)  # Should be [64, 1, 28, 28] for 64 images in a batch
    print("Target shape:", target.shape)  # Should be [64] for 64 labels
    break  # Just print the first batch shapes to check

Batch index: 0
Data shape: torch.Size([64, 1, 28, 28])
Target shape: torch.Size([64])


In [140]:
import torch
import torch.nn as nn

In [141]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.model = nn.Sequential(
            nn.Linear(in_features = 28 * 28, out_features = 256, bias = True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),  # Apply Dropout after the first hidden layer
            nn.Linear(in_features=256, out_features=128),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=128, out_features=10)
        )
        
    def forward(self, x):
        return self.model(x)

In [142]:
classifier = Classifier()

print(classifier)

Classifier(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): ReLU(inplace=True)
    (5): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [143]:
sum(params.numel() for params in classifier.parameters())

235146

In [144]:
# train_images = train_data.view(-1, 28 * 28)
# test_images = test_data.view(-1, 28 * 28)

In [145]:
# train_images.size(), test_images.size()

In [146]:
from torch.utils.data import DataLoader

In [147]:
# batch_size = 32
# train_dataloader = DataLoader(
#     dataset=list(zip(train_images, test_labels)),
#     batch_size = batch_size
# )
# valid_dataloader = DataLoader(
#     dataset=list(zip(test_images,test_labels)),
#     batch_size = batch_size
# )

In [148]:
data, labels = next(iter(train_loader))

data.size(), labels.size()

(torch.Size([64, 1, 28, 28]), torch.Size([64]))

In [149]:
# data, labels = next(iter(valid_dataloader))

# data.size(), labels.size()

In [150]:
import torch.optim as optim

learning_rate = 0.001
beta1 = 0.9
beat2 = 0.999

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(
    params=classifier.parameters(),
    lr=learning_rate,
    betas=(beta1, beat2)
)

criterion, optimizer

(CrossEntropyLoss(),
 Adam (
 Parameter Group 0
     amsgrad: False
     betas: (0.9, 0.999)
     capturable: False
     differentiable: False
     eps: 1e-08
     foreach: None
     fused: None
     lr: 0.001
     maximize: False
     weight_decay: 0
 ))

In [151]:
import numpy as np
from sklearn.metrics import accuracy_score

for epoch in range(100):
    train_loss = []
    valid_loss = []
    train_accuracy = []
    valid_accuracy = []
    
    # Training loop
    for (train_data, train_labels) in train_loader:
        train_data = train_data.view(-1, 28 * 28)
        # Forward pass
        predicted = classifier(train_data)
    
        # Compute the loss
        loss = criterion(predicted, train_labels)
        train_loss.append(loss.item())
        
        # Compute training accuracy
        predicted_classes = torch.argmax(predicted, axis=1)
        train_accuracy.append(accuracy_score(train_labels.cpu(), predicted_classes.cpu()))
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    # Validation loop
    for (valid_data, valid_labels) in test_loader:
        with torch.no_grad(): 
            valid_data = valid_data.view(-1, 28 * 28)
            predicted = classifier(valid_data)
            loss = criterion(predicted, valid_labels)
            valid_loss.append(loss.item())
            
            # Compute validation accuracy
            predicted_classes = torch.argmax(predicted, axis=1)
            valid_accuracy.append(accuracy_score(valid_labels.cpu(), predicted_classes.cpu()))
    
    # Print epoch results
    print("Epochs: [{}/{}] - train_loss: {:.4f} - valid_loss: {:.4f} - train_accuracy: {:.4f} - valid_accuracy: {:.4f}".format(
        epoch + 1, 10, np.mean(train_loss), np.mean(valid_loss), np.mean(train_accuracy), np.mean(valid_accuracy)
        )
    )

Epochs: [1/10] - train_loss: 0.3317 - valid_loss: 0.2153 - train_accuracy: 0.8986 - valid_accuracy: 0.9329
Epochs: [2/10] - train_loss: 0.1820 - valid_loss: 0.1729 - train_accuracy: 0.9450 - valid_accuracy: 0.9470
Epochs: [3/10] - train_loss: 0.1521 - valid_loss: 0.1523 - train_accuracy: 0.9526 - valid_accuracy: 0.9535
Epochs: [4/10] - train_loss: 0.1326 - valid_loss: 0.1331 - train_accuracy: 0.9586 - valid_accuracy: 0.9592
Epochs: [5/10] - train_loss: 0.1190 - valid_loss: 0.1340 - train_accuracy: 0.9630 - valid_accuracy: 0.9599
Epochs: [6/10] - train_loss: 0.1120 - valid_loss: 0.1276 - train_accuracy: 0.9638 - valid_accuracy: 0.9631
Epochs: [7/10] - train_loss: 0.1053 - valid_loss: 0.1213 - train_accuracy: 0.9667 - valid_accuracy: 0.9645
Epochs: [8/10] - train_loss: 0.0981 - valid_loss: 0.1250 - train_accuracy: 0.9687 - valid_accuracy: 0.9636
Epochs: [9/10] - train_loss: 0.0974 - valid_loss: 0.1192 - train_accuracy: 0.9693 - valid_accuracy: 0.9655
Epochs: [10/10] - train_loss: 0.0902 

KeyboardInterrupt: 

In [158]:
## Do the hyper parameter

def objective(trails):
    learning_rate = trails.suggest_float("lr", 0.0001, 0.001)
    epochs = trails.suggest_categorical("epochs", [5, 10, 15])
    
    model = Classifier()
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        params=model.parameters(),
        lr=learning_rate,
    )
    
    for epoch in range(epochs):
        train_loss = []
        valid_loss = []
        train_accuracy = []
        valid_accuracy = []
        
        # Training loop
        for (train_data, train_labels) in train_loader:
            train_data = train_data.view(-1, 28 * 28)
            # Forward pass
            predicted = classifier(train_data)
        
            # Compute the loss
            loss = criterion(predicted, train_labels)
            train_loss.append(loss.item())
            
            # Compute training accuracy
            predicted_classes = torch.argmax(predicted, axis=1)
            train_accuracy.append(accuracy_score(train_labels.cpu(), predicted_classes.cpu()))
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # Validation loop
        for (valid_data, valid_labels) in test_loader:
            with torch.no_grad(): 
                valid_data = valid_data.view(-1, 28 * 28)
                predicted = classifier(valid_data)
                loss = criterion(predicted, valid_labels)
                valid_loss.append(loss.item())
                
                # Compute validation accuracy
                predicted_classes = torch.argmax(predicted, axis=1)
                valid_accuracy.append(accuracy_score(valid_labels.cpu(), predicted_classes.cpu()))
        
        # Print epoch results
        print("Epochs: [{}/{}] - train_loss: {:.4f} - valid_loss: {:.4f} - train_accuracy: {:.4f} - valid_accuracy: {:.4f}".format(
            epoch + 1, 10, np.mean(train_loss), np.mean(valid_loss), np.mean(train_accuracy), np.mean(valid_accuracy)
            )
        )
        
    return np.mean(valid_loss)

In [159]:
import optuna

In [160]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2024-11-06 12:07:12,549] A new study created in memory with name: no-name-f4b4ab9d-d073-4d0c-a7ca-1fe88eccfed1


Epochs: [1/10] - train_loss: 0.0680 - valid_loss: 0.1225 - train_accuracy: 0.9781 - valid_accuracy: 0.9688
Epochs: [2/10] - train_loss: 0.0690 - valid_loss: 0.1209 - train_accuracy: 0.9784 - valid_accuracy: 0.9684
Epochs: [3/10] - train_loss: 0.0689 - valid_loss: 0.1238 - train_accuracy: 0.9779 - valid_accuracy: 0.9655
Epochs: [4/10] - train_loss: 0.0690 - valid_loss: 0.1218 - train_accuracy: 0.9783 - valid_accuracy: 0.9687
Epochs: [5/10] - train_loss: 0.0691 - valid_loss: 0.1239 - train_accuracy: 0.9783 - valid_accuracy: 0.9665
Epochs: [6/10] - train_loss: 0.0668 - valid_loss: 0.1190 - train_accuracy: 0.9781 - valid_accuracy: 0.9661
Epochs: [7/10] - train_loss: 0.0665 - valid_loss: 0.1259 - train_accuracy: 0.9788 - valid_accuracy: 0.9648
Epochs: [8/10] - train_loss: 0.0686 - valid_loss: 0.1177 - train_accuracy: 0.9783 - valid_accuracy: 0.9673
Epochs: [9/10] - train_loss: 0.0670 - valid_loss: 0.1125 - train_accuracy: 0.9783 - valid_accuracy: 0.9693
Epochs: [10/10] - train_loss: 0.0676 

[I 2024-11-06 12:10:50,763] Trial 0 finished with value: 0.11049108867442486 and parameters: {'lr': 0.000249463337907399, 'epochs': 15}. Best is trial 0 with value: 0.11049108867442486.


Epochs: [15/10] - train_loss: 0.0690 - valid_loss: 0.1105 - train_accuracy: 0.9778 - valid_accuracy: 0.9687
Epochs: [1/10] - train_loss: 0.0679 - valid_loss: 0.1237 - train_accuracy: 0.9782 - valid_accuracy: 0.9672
Epochs: [2/10] - train_loss: 0.0687 - valid_loss: 0.1148 - train_accuracy: 0.9783 - valid_accuracy: 0.9693
Epochs: [3/10] - train_loss: 0.0682 - valid_loss: 0.1204 - train_accuracy: 0.9786 - valid_accuracy: 0.9674
Epochs: [4/10] - train_loss: 0.0695 - valid_loss: 0.1149 - train_accuracy: 0.9782 - valid_accuracy: 0.9674


[I 2024-11-06 12:12:01,324] Trial 1 finished with value: 0.12406402679223635 and parameters: {'lr': 0.0002877132755124297, 'epochs': 5}. Best is trial 0 with value: 0.11049108867442486.


...

Epochs: [15/10] - train_loss: 0.0667 - valid_loss: 0.1188 - train_accuracy: 0.9782 - valid_accuracy: 0.9680
Epochs: [1/10] - train_loss: 0.0672 - valid_loss: 0.1273 - train_accuracy: 0.9790 - valid_accuracy: 0.9667
Epochs: [2/10] - train_loss: 0.0660 - valid_loss: 0.1206 - train_accuracy: 0.9783 - valid_accuracy: 0.9670
Epochs: [3/10] - train_loss: 0.0677 - valid_loss: 0.1182 - train_accuracy: 0.9776 - valid_accuracy: 0.9673
Epochs: [4/10] - train_loss: 0.0677 - valid_loss: 0.1203 - train_accuracy: 0.9783 - valid_accuracy: 0.9670
Epochs: [5/10] - train_loss: 0.0673 - valid_loss: 0.1315 - train_accuracy: 0.9786 - valid_accuracy: 0.9655
Epochs: [6/10] - train_loss: 0.0691 - valid_loss: 0.1128 - train_accuracy: 0.9782 - valid_accuracy: 0.9693
Epochs: [7/10] - train_loss: 0.0697 - valid_loss: 0.1076 - train_accuracy: 0.9777 - valid_accuracy: 0.9688
Epochs: [8/10] - train_loss: 0.0678 - valid_loss: 0.1234 - train_accuracy: 0.9783 - valid_accuracy: 0.9664
Epochs: [9/10] - train_loss: 0.0663 

[I 2024-11-06 13:02:45,102] Trial 18 finished with value: 0.12574179253410847 and parameters: {'lr': 0.0008200338680972635, 'epochs': 10}. Best is trial 6 with value: 0.10999692967509578.


Epochs: [10/10] - train_loss: 0.0692 - valid_loss: 0.1257 - train_accuracy: 0.9774 - valid_accuracy: 0.9680
Epochs: [1/10] - train_loss: 0.0662 - valid_loss: 0.1194 - train_accuracy: 0.9778 - valid_accuracy: 0.9668
Epochs: [2/10] - train_loss: 0.0669 - valid_loss: 0.1279 - train_accuracy: 0.9786 - valid_accuracy: 0.9688
Epochs: [3/10] - train_loss: 0.0683 - valid_loss: 0.1198 - train_accuracy: 0.9776 - valid_accuracy: 0.9652
Epochs: [4/10] - train_loss: 0.0689 - valid_loss: 0.1212 - train_accuracy: 0.9777 - valid_accuracy: 0.9678


[I 2024-11-06 13:04:02,714] Trial 19 finished with value: 0.12212316753735 and parameters: {'lr': 0.0003724324014636075, 'epochs': 5}. Best is trial 6 with value: 0.10999692967509578.


Epochs: [5/10] - train_loss: 0.0681 - valid_loss: 0.1221 - train_accuracy: 0.9779 - valid_accuracy: 0.9670


In [161]:
study.best_params

{'lr': 0.0006677854637103457, 'epochs': 15}

In [162]:
study.best_value

0.10999692967509578

In [163]:
study.best_trial

FrozenTrial(number=6, state=TrialState.COMPLETE, values=[0.10999692967509578], datetime_start=datetime.datetime(2024, 11, 6, 12, 22, 45, 226898), datetime_complete=datetime.datetime(2024, 11, 6, 12, 26, 16, 961547), params={'lr': 0.0006677854637103457, 'epochs': 15}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'lr': FloatDistribution(high=0.001, log=False, low=0.0001, step=None), 'epochs': CategoricalDistribution(choices=(5, 10, 15))}, trial_id=6, value=None)